# Introduction
In this exercise, a Natual Language Processing algorithm will be created using the Amazon Reviews dataset: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews?select=train.ft.txt.bz2

For this exercise, a sentiment analysis will be done.

In [37]:
%pip install langdetect

  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=f5c0c7666f0b83d8a45eac85780bd7937d503d87fe8b72ac6e0d9a7ce8b72dca
  Stored in directory: c:\users\ardejong\appdata\local\pip\cache\wheels\d1\c1\d9\7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [38]:
# Import the necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bz2
import re
from nltk.tokenize.regexp import regexp_tokenize
from langdetect import detect

In [13]:
train_file = bz2.BZ2File("train.ft.txt.bz2")
test_file = bz2.BZ2File("test.ft.txt.bz2")

# Load and decode
lines_train = [x.decode('utf-8') for x in train_file.readlines()]
lines_test = [x.decode('utf-8') for x in test_file.readlines()]

# Split in two: sentiment and review
score_review_list_train = [re.split("\s+", l.strip('__label__'), 1) for l in lines_train]
score_review_list_test = [re.split("\s+", l.strip('__label__'), 1) for l in lines_test]

train = pd.DataFrame(score_review_list_train, columns = ['score', 'review'] )
test = pd.DataFrame(score_review_list_test, columns = ['score', 'review'] )

train.head()

,score,review
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."


In [28]:
#train['n_tokens'] = [len(regexp_tokenize(line, '\w+')) for line in train.review]
test['n_tokens'] = [len(regexp_tokenize(line, '\w+')) for line in test.review]

test.head()

,score,review,n_tokens
0,2,Great CD: My lovely Pat has one of the GREAT v...,107
1,2,One of the best game music soundtracks - for a...,152
2,1,Batteries died within a year ...: I bought thi...,59
3,2,"works fine, but Maha Energy is better: Check o...",39
4,2,Great for the non-audiophile: Reviewed quite a...,73


In [ ]:
languages = []

for i in train.review:
    try:
        lang = detect(i)
    except:
        lang = "not_detected"
    languages.append(lang)

train['language'] = languages

train.head()

In [45]:
languages = []

for i in test.review:
    try:
        lang = detect(i)
    except:
        lang = "not_detected"
    languages.append(lang)

    

test['language'] = languages

test.head()

,score,review,n_tokens,language
0,2,Great CD: My lovely Pat has one of the GREAT v...,107,en
1,2,One of the best game music soundtracks - for a...,152,en
2,1,Batteries died within a year ...: I bought thi...,59,en
3,2,"works fine, but Maha Energy is better: Check o...",39,en
4,2,Great for the non-audiophile: Reviewed quite a...,73,en


In [47]:
print(test.language.unique())

['en' 'es' 'so' 'it' 'hu' 'cy' 'fr' 'pt' 'de' 'ca' 'sl' 'nl' 'id' 'af'
 'sv' 'pl' 'vi' 'da' 'et' 'hr' 'sq' 'no' 'tr' 'tl' 'not_detected']
